# This is the first step in the pipeline
### Spots are detected in this notebook. The input file is expected to be in the zarr format 

In [1]:
import pandas as pd
import time
import os
import sys
import zarr
import napari 
import dask.array as da 

pythonPackagePath = os.path.abspath('../src/')
sys.path.append(pythonPackagePath)
from parallel import Detector
from gaussian_visualization import visualize_3D_gaussians

### Do not change the code in cell below 

In [2]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
# base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'movie_data')
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'test_movie_1')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

save_directory = 'datasets'
save_directory_full = os.path.join(base_dir, save_directory)

## Follow the Instructions below to run through the notebook properly 

The purpose of this notebook is to perform spot detection on your full movie. The movie is expected to be a 3 channel movie which is saved as a zarr object. If your movie is not a zarr object you can convert it to a zarr object by running and following the steps provided to you under Final/Data Preparation/full_movie_to_zarr.ipynb

**For Initialising the Detector Object** 

1. Detector object is the main object for which you will setup the parameters to work in this notebook. 
2. **zarr_obj**: is the object which efficiently stores the movie 
3. **save_directory**: is fixed and does not need to be changed. However, for reference the files from this notebook will be outputted and saved in Final/movie_data/datasets directory 
4. **spot_intensity**: This is the minimum intensity which a spot will have in your movie. Anything below this can be called noise/background. You can determine the spot_intensity using fiji or napari to determine minimum bright spots which are of interest. If you set a spot_intensity too high very few spots will be detected, however, if you set a intensity too low a lot of spots including a lot of noise will be detected. 
5. **dist_between_spots**: this distance divided by 2 is the minimum distance that should exist between spots in pixels. For example if you set this to 10 then all spots within 5 pixels of the center of your spot will be dropped(to understand which spot is dropped you can refer to the source code in the Final/src/gaussian_fitting.py file)
6. **sigma_estimations**: This is the spread/radius of your spots from the center. It is entered in pixels and follows the order [z,y,x]. If you expect your spot to have a radius of 4 in z and 2 in x and y then you should enter [4,2,2]. To determine the spread/radius of your spot you can visualise in fiji and look at the metadata to understand the pixel radius. 
7. **quality_threshold_mu**: This is a threshold that rejects detections with a large error in the gaussian fit for the spot center, averaged in xyz. default value is 1 pixel.
8. **quality_threshold_sigma**: This is a threshold that rejects detections with a large error in the gaussian fit for the spot size, averaged in xyz. default value is 2.5 pixels.
9. **n_jobs**: Detector class allows for parallel processing and the number of cores you want to use can be determined here. You can set it to -1 and it will use all_cores - 1 for processing. It is important to allow for parallel processing else for larger movies it will take a lot of time. 
10. **channel_to_detect**: The number of channel to detect. Convention is 1 for channel 1, 2 for channel 2 and 3 for channel 3. The detector object can only detect one channel at a time. 

**For running processing on frames** (run_parallel_frame_processing)

1. **max_frames**: the maximum frames to process. This can be useful when you just want to test your parameters selected for the Detector object like spot_intensity, dist_between_spots and sigma_estimates. 
2. **all_frames**: If all frames is set to True then all frames are processed and the **max_frames** parameter is ignored. It is recommended to initially decide all the parameters on a subset of frames and then move onto this step as it may take a lot of time for larger movies. 


**Note**

-> Cores to be utilized can be increased as available. Keep in mind that limitation can be posed by the RAM of your machine. As more cores are utilized more RAM is needed. 

-> Detection can be only performed on 1 channel at a time


## Set all parameters in the below cell 

In [3]:
#refer to the above cell for explanation of each parameter 
spot_intensity = 180
dist_between_spots = 10
sigma_estimations = [4,2,2]
quality_threshold_mu = 1
quality_threshold_sigma = 2.5
n_jobs = -1
channel_to_detect = 3 
max_frames = 2 
all_frames = False

In [4]:
#Import the zarr file by adding file path in read mode
z2 = zarr.open(zarr_full_path, mode='r')

In [5]:
z2.info

Type,zarr.core.Array
Data type,uint16
Shape,"(130, 3, 75, 150, 275)"
Chunk shape,"(1, 1, 75, 150, 275)"
Order,C
Read-only,True
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,2413125000 (2.2G)
No. bytes stored,656696875 (626.3M)
Storage ratio,3.7


In [6]:
frames = z2.shape[0]
print(f'the number of frames are {frames}')

the number of frames are 130


## In the below cell Detector object is initilized to perform detection. More details on the Detector object can be attained by the following line of code: 
**copy and paste in a new cell**

?Detector

In [9]:
detector = Detector(zarr_obj = z2, 
                    save_directory = save_directory_full, 
                    spot_intensity = spot_intensity, 
                    dist_between_spots = dist_between_spots, 
                    sigma_estimations = sigma_estimations, n_jobs = n_jobs, channel_to_detect = channel_to_detect)

In [10]:
#the following function returns the dataframe and also saves it to the provided path in pkl format
#set all_frames = True, to process all the time frames 
#max_frames is useful when you just want to perform detection on a subset of frames. 
#Note: when all_frames= True then max_frames is ignored 
df = detector.run_parallel_frame_processing(max_frames = max_frames, all_frames = all_frames)

Processing frames: 100%|██████████| 2/2 [00:03<00:00,  1.65s/it]

the number of times the gaussian fitting worked was 311 and the number of times the gaussian did not fit was 0
the number of times the gaussian fitting worked was 328 and the number of times the gaussian did not fit was 0


In [12]:
# filter out detections that don't pass the quality thresholdq

# for each line in test, calculate the mean of errors[1] and errors[2]
# if the mean of errors[1] is greater than mu_threshold or mean of errors[2] is greater than sigma_threshold, then keep it in the dataframe test_2

df['mean_errors_mu'] = df['errors'].apply(lambda x: pd.Series(x[1]).mean())
df['mean_errors_sigma'] = df['errors'].apply(lambda x: pd.Series(x[2]).mean())

# save the rejected detections to a new dataframe
df_rejected = df[(df['mean_errors_mu'] > 1) & (df['mean_errors_sigma'] >= 2.5)]

# filter out the rejected detections from the original dataframe
df = df[(df['mean_errors_mu'] <= 1) & (df['mean_errors_sigma'] < 2.5)]
print(f'Number of detections retained: {len(df)}')
print(f'Number of detections filtered out: {len(df_rejected)}')


Number of detections retained: 518
Number of detections filtered out: 20


# Visualising the Output
## Labels are only for time frame 0, for all z slices 

In [13]:
masks_accepted = visualize_3D_gaussians(zarr_obj = z2, gaussians_df = df)
masks_rejected = visualize_3D_gaussians(zarr_obj = z2, gaussians_df = df_rejected)

# masks = visualize_3D_gaussians(zarr_obj = z2, gaussians_df = df)

### Below you can see detected spots as masks on the original image and can adjust detection parameters if needed 

In [17]:
dask_array

dask.array<from-zarr, shape=(130, 3, 75, 150, 275), dtype=uint16, chunksize=(1, 1, 75, 150, 275), chunktype=numpy.ndarray>

In [23]:
detection_channel[2]

dask.array<getitem, shape=(75, 150, 275), dtype=uint16, chunksize=(75, 150, 275), chunktype=numpy.ndarray>

In [31]:
# Create a napari viewer
viewer = napari.Viewer()

#access channel 3 only from zarr array 
dask_array = da.from_zarr(z2)

#the axis arrangement is (t,c,z,y,x)
# importing the channel_to_detect
detection_channel = dask_array[:,:,:,:,:]

# which channel to show
visibility_mask = [False, False, False]
visibility_mask[channel_to_detect-1] = True

# Add the 4D stack to the viewer
# Can change the names of the channels as needed
layer_raw = viewer.add_image(detection_channel, channel_axis = 1, name = ['channel 1', 'channel 2', 'channel 3'], interpolation3d = 'nearest', blending = 'additive', colormap = 'magenta', visible = visibility_mask)
# layer_raw = viewer.add_image(detection_channel, channel_axis = 1, name = ['detection channel'], interpolation3d = 'nearest', blending = 'additive', colormap = 'magenta')

# layer_mask = viewer.add_image(masks, name = 'detections mask')
layer_mask_accepted = viewer.add_image(masks_accepted, name = 'accepted detections', interpolation3d = 'nearest', blending = 'additive', colormap = 'green')
layer_mask_rejected = viewer.add_image(masks_rejected, name = 'rejected detections', interpolation3d = 'nearest', blending = 'additive', colormap = 'cyan', visible = False)

#other useful parameters 
#color_map = list
#contrast_limits = list of list 

# Add Bounding Box
layer_raw[0].bounding_box.visible = True
layer_raw[1].bounding_box.visible = True
layer_raw[2].bounding_box.visible = True

If the detections don't line up well with the spots in the image:
* make sure you are looking at the first time point
* mouse over the spots in napari to get a sense for the intensity of the spots vs background - use the threshold distinguishing spots from background as spot_intensity 
* increase or lower the quality_threshold_mu and quality_threshold_sigma depending on whether there are too many or too few detections compared to the number of visible spots
* if those don't work, then vary the distance_between_spots and sigma_estimations based on the density and size of spots

# move to 02.filtering_spots for next steps 